In [ ]:
# Montar o google drive no coolab}
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import classification_report, ConfusionMatrixDisplay, confusion_matrix, f1_score
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer

## Importando dataframes

In [ ]:
df_teste = pd.read_pickle('/content/drive/MyDrive/Mestrado/Dissertação/Experimentos/anxiety/01. data/01. raw/sbr_ansiedade_test.pkl')

In [ ]:
df_teste.head()

In [ ]:
print('teste:', df_teste.shape)

# Tratamento

In [ ]:
df_teste.reset_index(inplace=True)

In [ ]:
df_teste['Numeric_Diagnosed_YN'] = df_teste['Diagnosed_YN'].map({'no': 0, 'yes': 1})

In [ ]:
# Faço a contagem de posts
def contar_posts(texto):
    return len(texto)

df_teste['Qt_Messages'] = df_teste['Text'].apply(contar_posts)

In [ ]:
def min_timeline(lista):
    return lista[- menor_timeline:]

menor_timeline = df_teste['Qt_Messages'].min()
df_teste['label_min'] = df_teste['1to10_label'].apply(min_timeline)

# Testes para obtenção da solução final

Nesta seção estão os testes de janela e corte para a solução

### Funções para cálculo da métrica f1-*latency*

Para cálculo da métrica de interesse f1-*latency* são necessárias algumas etapas representadas pelas funções abaixo.

In [ ]:
def value_p(k):
    """Get the penalty value for the F latency measure.

    Parameters
    ----------
    k : int
        Median number of posts from the positive users.

    Returns
    -------
    penalty : float
        Penalty to use.
    """
    return -(np.log(1 / 3) / (k - 1))


def f_penalty(k, _p):
    """Get the penalty of the current user delay.

    Parameters
    ----------
    k : int
        Current user delay.
    _p : float
        Penalty.

    Returns
    -------
    f_penalty : float
        Penalty latency.
    """
    return -1 + (2 / (1 + np.exp((-_p) * (k - 1))))


def speed(y_pred, y_true, d, p):
    """Get speed for every user correctly classified as positive."""
    penalty_list = [
        f_penalty(k=d[i], _p=p)
        for i in range(len(y_pred))
        if y_pred[i] == 1 and y_true[i] == 1
    ]

    if len(penalty_list) != 0:
        return 1 - np.median(penalty_list)
    else:
        return 0.0


def f_latency(labels, true_labels, delays, penalty):
    """F latency metric.

    Metric proposed by Sadeque and others in [1]_.

    Parameters
    ----------
    labels : list of int
        Predicted label for each user.
    true_labels : list of int
        True label for each user.
    delays : list of int
        Decision delay for each user.
    penalty : float
        Penalty. Defines how quickly the penalty should increase.

    Returns
    -------
    f_latency_metric : float
        F latency measure.

    References
    ----------
    .. [1] `Sadeque, F., Xu, D., & Bethard, S. (2018, February). Measuring the
        latency of depression detection in social media. In Proceedings of the
        Eleventh ACM International Conference on Web Search and Data Mining
        (pp. 495-503).`_
    """
    f1_score = metrics.f1_score(y_pred=labels, y_true=true_labels, average="binary")
    speed_value = speed(y_pred=labels, y_true=true_labels, d=delays, p=penalty)

    return f1_score * speed_value

### Remoção de valores
Função auxiliar para remoção de valores zero e um da lista de notas

In [ ]:
def remove(lista, lista_valores):
    """
    Remove zeros de uma lista.

    Parâmetros:
    lista (list): A lista de entrada da qual os zeros serão removidos.

    Retorna:
    list: Uma nova lista sem os zeros.
    """
    return [item for item in lista if item not in lista_valores]

### Treinando modelo
Realizando treino do modelo e obtendo as métricas de performance

In [ ]:
def treino_modelo(df, janela, corte, metrica='media', target='label', retirar_notas=[]):
    """
    Treina o modelo com os pontos de corte e janela a partir de uma métrica escolhida

    Parâmetros:
    df: DataFrame de treinamento
    janela: quantidade de mensagens a serem consideradas na janela móvel
    corte: valor de threshold de corte
    métrica: métrica utilizada como critério (media, mediana ou soma)

    Retorna:
    df_metrics: DataFrame contendo as métricas de performance do treinamento
    """
    # Inicializando o dataframe de métricas
    df_metrics = pd.DataFrame(columns=['window', 'cut', 'f1_latency', 'pos_f1', 'neg_f1',
                                       'wheighted_avg_f1', 'macro_precision', 'macro_recall', 'macro_f1'])

    # Tratando os dados e calculando somas e médias janeladas
    coluna_numerica = []
    coluna_janela_metrica = []

    for row in df[target]:
        lista_numerica = []
        for item in row:
            try:
                objeto = int(item)
                if objeto > 10:
                    objeto = 10
            except:
                objeto = 0
            lista_numerica.append(objeto)
        coluna_numerica.append(lista_numerica)

        # Métrica móvel
        metrica_movel = []
        for i in range(len(lista_numerica) - janela + 1):
            janela_movel = lista_numerica[i:i + janela]
            janela_movel = remove(janela_movel, retirar_notas)

            if metrica == 'media':
              valor_metrica = sum(janela_movel) / janela
            elif metrica == 'mediana':
              valor_metrica = statistics.median(janela_movel)
            elif metrica == 'soma':
              valor_metrica = sum(janela_movel)
            metrica_movel.append(valor_metrica)
        coluna_janela_metrica.append(metrica_movel)

    # Inserindo as colunas criadas
    df['tl_prediction_numeric'] = coluna_numerica
    df[f'{metrica}_movel_prediction'] = coluna_janela_metrica

    # Obtendo a predição e calculando a quantidade de mensagens necessárias para isso
    coluna_msgs_lidas = []
    coluna_respostas = []
    for i in range(len(df)):
        resposta = 0
        lista_prov = []
        for j in df[f'{metrica}_movel_prediction'][i]:
            if j < corte:
                lista_prov.append(j)
            elif j >= corte:
                lista_prov.append(j)
                resposta = 1
                break

        n_msgs = len(lista_prov) + (janela-1)

        coluna_msgs_lidas.append(n_msgs)
        coluna_respostas.append(resposta)

    # Inserindo no dataframe
    df['predict_Diagnosed_YN'] = coluna_respostas
    df['qt_msgs_lidas'] = coluna_msgs_lidas

    # Calculando métricas de performance
    y_train = df['Numeric_Diagnosed_YN']
    y_pred = df['predict_Diagnosed_YN']

    report = classification_report(y_pred, y_train, output_dict=True)

    macro_precision = report['macro avg']['precision']
    macro_recall = report['macro avg']['recall']
    macro_f1 = report['macro avg']['f1-score']

    pos_f1 = report['1']['f1-score']
    neg_f1 = report['0']['f1-score']
    wheighted_avg_f1 = report['weighted avg']['f1-score']

    # Medida f1-latency
    f1_latency = f_latency(df['predict_Diagnosed_YN'], df['Numeric_Diagnosed_YN'], df['qt_msgs_lidas'], 0.0078)

    dict_to_append = {'window': janela, 'cut': corte, 'f1_latency': f1_latency, 'pos_f1': pos_f1, 'neg_f1': neg_f1,
                      'wheighted_avg_f1': wheighted_avg_f1, 'macro_precision': macro_precision, 'macro_recall': macro_recall, 'macro_f1': macro_f1}

    df_metrics = pd.concat([df_metrics, pd.DataFrame([dict_to_append])], ignore_index=True)

    return df_metrics, df

## Desenvolvimento da solução


### Corpus de teste

#### Métricas de performance

In [ ]:
janela = 15
corte = 2.5
df_metrics_teste, df_teste = treino_modelo(df_teste, janela, corte, target='label_min', retirar_notas=[0, 1, 2])
df_metrics_teste

In [ ]:
df_metrics_teste.sort_values('f1_latency', ascending=False).head()

In [ ]:
df_metrics_teste.sort_values('macro_f1', ascending=False).head()

In [ ]:
df_metrics_teste.sort_values('pos_f1', ascending=False).head()

In [ ]:
# Confusion matrix
cm = confusion_matrix(df_teste['Numeric_Diagnosed_YN'], df_teste['predict_Diagnosed_YN'])

# Exibindo a matriz de confusão
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted 0', 'Predicted 1'], yticklabels=['Actual 0', 'Actual 1'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
df_metrics_teste.to_csv('/content/drive/MyDrive/Mestrado/Dissertação/Experimentos/anxiety/04. results/GPTrelev_metricas_teste_MinTL.csv', index=False)
df_teste.to_pickle('/content/drive/MyDrive/Mestrado/Dissertação/Experimentos/anxiety/01. data/03. prediction/prediction_GPTrelev_MinTL.pkl')

#### Tempo de antecipação
Neste seção vou calcular quanto tempo em média houve de antecipação do risco

In [ ]:
df_teste_clean = pd.read_pickle('/content/drive/MyDrive/Mestrado/Dissertação/Experimentos/anxiety/01. data/02. clean/test_clean_corpus.pkl')

In [ ]:
df_teste = pd.read_pickle('/content/drive/MyDrive/Mestrado/Dissertação/Experimentos/anxiety/01. data/03. prediction/prediction_GPTrelev_MinTL.pkl')

In [ ]:
df_teste_merged = pd.merge(df_teste, df_teste_clean[['User_ID', 'List_Timeline']], on='User_ID', how='left')

In [ ]:
from datetime import datetime

# Função para converter a lista de datas em objetos datetime
def converter_datas(lista_datas_str):
    # Remover os colchetes e espaços em branco
    # lista_datas_str = lista_datas_str.strip('[]').replace("'", "").split(', ')
    # Converter cada data para datetime
    return [pd.to_datetime(data_str, format='%d-%b-%Y %H:%M') for data_str in lista_datas_str]


# Função para calcular a diferença em dias entre duas datas
def calcular_diferenca_dias(data1, data2):
    delta = data1 - data2
    return delta.days

# Aplicando a conversão de datas ao DataFrame
df_teste_merged['List_Timeline'] = df_teste_merged['List_Timeline'].apply(converter_datas)

# Aplicando a função ao DataFrame
df_teste_merged['ultima_data_tweet_lido'] = df_teste_merged.apply(lambda row: row['List_Timeline'][row['qt_msgs_lidas'] - 1], axis=1)
df_teste_merged['ultima_data_tweet_lista'] = df_teste_merged.apply(lambda row: row['List_Timeline'][-1], axis=1)
df_teste_merged['dias_antecipados'] = df_teste_merged.apply(lambda row: calcular_diferenca_dias(row['ultima_data_tweet_lista'], row['ultima_data_tweet_lido']), axis=1)

df_teste_merged.head()

In [ ]:
dias_antecipacao_geral = df_teste_merged['dias_antecipados'].mean()
dias_antecipacao_pos = df_teste_merged[df_teste_merged['Numeric_Diagnosed_YN']==1]['dias_antecipados'].mean()
print(f'Média de dias de antecipação de risco geral: {dias_antecipacao_geral}')
print(f'Média de dias de antecipação de risco da classe positiva: {dias_antecipacao_pos}')

In [ ]:
df_teste_merged.to_csv('/content/drive/MyDrive/Mestrado/Dissertação/Experimentos/gpt-3/1-10_relevancia/prediction_GPTrelev_MinTL.csv', index=False)

#### Estatística descritiva das classificações de tweets

In [ ]:
df_teste = pd.read_csv('/content/drive/MyDrive/Mestrado/Dissertação/Experimentos/gpt-3/1-10_relevancia/prediction_GPTrelev_MinTL.csv')
df_teste.head()

In [ ]:
import ast
from collections import Counter

# Tratando os dados de teste
# Transforma a coluna de string para lista
# df_teste['label_min'] = df_teste['label_min'].apply(ast.literal_eval)

# Função para substituir valores não numéricos por 0
def limpar_lista(lista):
    nova_lista = []
    for item in lista:
        try:
            valor = int(item)
            if valor > 10:
              nova_lista.append(10)
            else:
              nova_lista.append(valor)
        except ValueError:
            nova_lista.append(0)  # Substitui por 0 se não for possível converter para inteiro
    return nova_lista

# Aplicar a função para limpar as listas na coluna 'label_min'
df_teste['label_min'] = df_teste['label_min'].apply(limpar_lista)

counter = Counter()
# Iterar sobre cada lista da coluna 'label_min' e atualizar o Counter com as contagens de cada valor
for lista in df_teste['label_min']:
    counter.update(lista)

In [ ]:
counter

In [ ]:
# Converter o Counter para um DataFrame do pandas para facilitar a plotagem
histogram_df = pd.DataFrame.from_dict(counter, orient='index').reset_index()
histogram_df.columns = ['Número', 'Contagem']

# Ordenar os números
histogram_df = histogram_df.sort_values(by='Número')

# Plotar o histograma
plt.figure(figsize=(10, 6))
plt.bar(histogram_df['Número'], histogram_df['Contagem'], color='lightgrey')
plt.xlabel('Pontuação de risco')
plt.ylabel('Quantidade')
plt.xticks(range(11))
plt.show()

In [ ]:
histogram_df['percentual'] = (histogram_df['Contagem'] / histogram_df['Contagem'].sum()) * 100
histogram_df